import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [4]:
pip install pmdarima

   ---------------------------------------- 0.0/614.7 kB ? eta -:--:--
   --------------------------------------  614.4/614.7 kB 19.5 MB/s eta 0:00:01
   --------------------------------------- 614.7/614.7 kB 12.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.8 MB ? eta -:--:--
   --------------------------- ------------ 1.9/2.8 MB 61.1 MB/s eta 0:00:01
   ---------------------------------------  2.8/2.8 MB 59.0 MB/s eta 0:00:01
   ---------------------------------------- 2.8/2.8 MB 35.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/9.9 MB ? eta -:--:--
   ------ --------------------------------- 1.7/9.9 MB 36.2 MB/s eta 0:00:01
   -------------- ------------------------- 3.6/9.9 MB 38.5 MB/s eta 0:00:01
   ----------------- ---------------------- 4.2/9.9 MB 38.0 MB/s eta 0:00:01
   ----------------------- ---------------- 5.8/9.9 MB 28.4 MB/s eta 0:00:01
   -------------------------------- ------- 8.0/9.9 MB 31.9 MB/s eta 0:00:01
   ------------

**SARIMA**

In [2]:
import pandas as pd
import numpy as np
from pmdarima import auto_arima
from sklearn.metrics import mean_squared_error, mean_absolute_error
import warnings
import gc

warnings.filterwarnings("ignore")

# Load your dataset (assuming it is a CSV file)
df = pd.read_csv('phase1_only_records.csv')

# Convert 'Date' and 'Hour' columns to a datetime object
df['Datetime'] = pd.to_datetime(df['Date']) + pd.to_timedelta(df['Hour'], unit='h')

# Set 'Datetime' as index
df.set_index('Datetime', inplace=True)
exog_vars = ['Temperature']

# Prepare a function to fit and forecast using auto_arima for each ID in chunks
def fit_forecast_sarima(df, split_ratio=0.8, chunk_size=50):
    ids = df['ID'].unique()
    forecasts = []
    
    for chunk_start in range(0, len(ids), chunk_size):
        chunk_end = min(chunk_start + chunk_size, len(ids))
        chunk_ids = ids[chunk_start:chunk_end]
        
        for i, id_ in enumerate(chunk_ids, start=chunk_start+1):
            print(f"{i} Processing ID: {id_}")
            df_id = df[df['ID'] == id_].sort_index()

            # Split the data into train and test
            split_index = int(len(df_id) * split_ratio)
            train = df_id.iloc[:split_index]
            test = df_id.iloc[split_index:]

            # Fit the auto_arima model
            model = auto_arima(train['Demand_kWh'],
                               exogenous=train[exog_vars],
                               seasonal=True,
                               m=24,  # Assuming daily seasonality with hourly data
                               stepwise=True,
                               suppress_warnings=True,
                               trace=False,
                               max_p=2, max_q=2, max_P=1, max_Q=1, max_order=5)

            # Forecast
            n_periods = len(test)
            forecast = model.predict(n_periods=n_periods, exogenous=test[exog_vars])

            # Store the forecast and actual values
            forecast_df = pd.DataFrame({
                'ID': id_,
                'Datetime': test.index,
                'Actual_Demand': test['Demand_kWh'],
                'Predicted_Demand': forecast
            })

            forecasts.append(forecast_df)
            
        # Run garbage collection to free memory
        gc.collect()
    
    return pd.concat(forecasts)

# Run the forecasting function
results_df = fit_forecast_sarima(df)

# Calculate overall error metrics
mse = mean_squared_error(results_df['Actual_Demand'], results_df['Predicted_Demand'])
mae = mean_absolute_error(results_df['Actual_Demand'], results_df['Predicted_Demand'])
rmse = np.sqrt(mse)

print(f"Overall MSE: {mse}")
print(f"Overall MAE: {mae}")
print(f"Overall RMSE: {rmse}")

# Write the results to a file
results_df['Date'] = results_df['Datetime'].dt.date
results_df['Hour'] = results_df['Datetime'].dt.hour
results_df = results_df[['ID', 'Date', 'Hour', 'Actual_Demand', 'Predicted_Demand']]
results_df.to_csv('sarima_results_1_8020.csv', index=False)

print("Forecast results have been written to forecast_results_2_8020.csv")


1 Processing ID: Exp_553
2 Processing ID: Exp_401
3 Processing ID: Exp_266
4 Processing ID: Exp_62
5 Processing ID: Exp_111
6 Processing ID: Exp_567
7 Processing ID: Exp_570
8 Processing ID: Exp_500
9 Processing ID: Exp_52
10 Processing ID: Exp_222
11 Processing ID: Exp_587
12 Processing ID: Exp_17
13 Processing ID: Exp_462
14 Processing ID: Exp_271
15 Processing ID: Exp_49
16 Processing ID: Exp_451
17 Processing ID: Exp_502
18 Processing ID: Exp_86
19 Processing ID: Exp_612
20 Processing ID: Exp_509
21 Processing ID: Exp_12
22 Processing ID: Exp_208
23 Processing ID: Exp_529
24 Processing ID: Exp_128
25 Processing ID: Exp_56
26 Processing ID: Exp_635
27 Processing ID: Exp_93
28 Processing ID: Exp_313
29 Processing ID: Exp_446
30 Processing ID: Exp_643
31 Processing ID: Exp_223
32 Processing ID: Exp_315
33 Processing ID: Exp_649
34 Processing ID: Exp_191
35 Processing ID: Exp_445
36 Processing ID: Exp_518
37 Processing ID: Exp_365
38 Processing ID: Exp_356
39 Processing ID: Exp_420
40 

In [3]:
import pandas as pd
import numpy as np
from pmdarima import auto_arima
from sklearn.metrics import mean_squared_error, mean_absolute_error
import warnings
import gc

warnings.filterwarnings("ignore")

# Load your dataset (assuming it is a CSV file)
df = pd.read_csv('phase1_only_records.csv')

# Convert 'Date' and 'Hour' columns to a datetime object
df['Datetime'] = pd.to_datetime(df['Date']) + pd.to_timedelta(df['Hour'], unit='h')

# Set 'Datetime' as index
df.set_index('Datetime', inplace=True)
exog_vars = ['Temperature']

# Prepare a function to fit and forecast using auto_arima for each ID in chunks
def fit_forecast_sarima(df, split_ratio=0.75, chunk_size=50):
    ids = df['ID'].unique()
    forecasts = []
    
    for chunk_start in range(0, len(ids), chunk_size):
        chunk_end = min(chunk_start + chunk_size, len(ids))
        chunk_ids = ids[chunk_start:chunk_end]
        
        for i, id_ in enumerate(chunk_ids, start=chunk_start+1):
            print(f"{i} Processing ID: {id_}")
            df_id = df[df['ID'] == id_].sort_index()

            # Split the data into train and test
            split_index = int(len(df_id) * split_ratio)
            train = df_id.iloc[:split_index]
            test = df_id.iloc[split_index:]

            # Fit the auto_arima model
            model = auto_arima(train['Demand_kWh'],
                               exogenous=train[exog_vars],
                               seasonal=True,
                               m=24,  # Assuming daily seasonality with hourly data
                               stepwise=True,
                               suppress_warnings=True,
                               trace=False,
                               max_p=2, max_q=2, max_P=1, max_Q=1, max_order=5)

            # Forecast
            n_periods = len(test)
            forecast = model.predict(n_periods=n_periods, exogenous=test[exog_vars])

            # Store the forecast and actual values
            forecast_df = pd.DataFrame({
                'ID': id_,
                'Datetime': test.index,
                'Actual_Demand': test['Demand_kWh'],
                'Predicted_Demand': forecast
            })

            forecasts.append(forecast_df)
            
        # Run garbage collection to free memory
        gc.collect()
    
    return pd.concat(forecasts)

# Run the forecasting function
results_df = fit_forecast_sarima(df)

# Calculate overall error metrics
mse = mean_squared_error(results_df['Actual_Demand'], results_df['Predicted_Demand'])
mae = mean_absolute_error(results_df['Actual_Demand'], results_df['Predicted_Demand'])
rmse = np.sqrt(mse)

print(f"Overall MSE: {mse}")
print(f"Overall MAE: {mae}")
print(f"Overall RMSE: {rmse}")

# Write the results to a file
results_df['Date'] = results_df['Datetime'].dt.date
results_df['Hour'] = results_df['Datetime'].dt.hour
results_df = results_df[['ID', 'Date', 'Hour', 'Actual_Demand', 'Predicted_Demand']]
results_df.to_csv('sarima_results_1_7525.csv', index=False)

print("Forecast results have been written to forecast_results_2_8020.csv")


1 Processing ID: Exp_553
2 Processing ID: Exp_401
3 Processing ID: Exp_266
4 Processing ID: Exp_62
5 Processing ID: Exp_111
6 Processing ID: Exp_567
7 Processing ID: Exp_570
8 Processing ID: Exp_500
9 Processing ID: Exp_52
10 Processing ID: Exp_222
11 Processing ID: Exp_587
12 Processing ID: Exp_17
13 Processing ID: Exp_462
14 Processing ID: Exp_271
15 Processing ID: Exp_49
16 Processing ID: Exp_451
17 Processing ID: Exp_502
18 Processing ID: Exp_86
19 Processing ID: Exp_612
20 Processing ID: Exp_509
21 Processing ID: Exp_12
22 Processing ID: Exp_208
23 Processing ID: Exp_529
24 Processing ID: Exp_128
25 Processing ID: Exp_56
26 Processing ID: Exp_635
27 Processing ID: Exp_93
28 Processing ID: Exp_313
29 Processing ID: Exp_446
30 Processing ID: Exp_643
31 Processing ID: Exp_223
32 Processing ID: Exp_315
33 Processing ID: Exp_649
34 Processing ID: Exp_191
35 Processing ID: Exp_445
36 Processing ID: Exp_518
37 Processing ID: Exp_365
38 Processing ID: Exp_356
39 Processing ID: Exp_420
40 

In [3]:
import pandas as pd
import numpy as np
from pmdarima import auto_arima
from sklearn.metrics import mean_squared_error, mean_absolute_error
import warnings
import gc

warnings.filterwarnings("ignore")

# Load your dataset (assuming it is a CSV file)
df = pd.read_csv('phase1_only_records.csv')

# Convert 'Date' and 'Hour' columns to a datetime object
df['Datetime'] = pd.to_datetime(df['Date']) + pd.to_timedelta(df['Hour'], unit='h')

# Set 'Datetime' as index
df.set_index('Datetime', inplace=True)
exog_vars = ['Temperature']

# Prepare a function to fit and forecast using auto_arima for each ID in chunks
def fit_forecast_sarima(df, split_ratio=0.5, chunk_size=50):
    ids = df['ID'].unique()
    forecasts = []
    
    for chunk_start in range(0, len(ids), chunk_size):
        chunk_end = min(chunk_start + chunk_size, len(ids))
        chunk_ids = ids[chunk_start:chunk_end]
        
        for i, id_ in enumerate(chunk_ids, start=chunk_start+1):
            print(f"{i} Processing ID: {id_}")
            df_id = df[df['ID'] == id_].sort_index()

            # Split the data into train and test
            split_index = int(len(df_id) * split_ratio)
            train = df_id.iloc[:split_index]
            test = df_id.iloc[split_index:]

            # Fit the auto_arima model
            model = auto_arima(train['Demand_kWh'],
                               exogenous=train[exog_vars],
                               seasonal=True,
                               m=24,  # Assuming daily seasonality with hourly data
                               stepwise=True,
                               suppress_warnings=True,
                               trace=False,
                               )

            # Forecast
            n_periods = len(test)
            forecast = model.predict(n_periods=n_periods, exogenous=test[exog_vars])

            # Store the forecast and actual values
            forecast_df = pd.DataFrame({
                'ID': id_,
                'Datetime': test.index,
                'Actual_Demand': test['Demand_kWh'],
                'Predicted_Demand': forecast
            })

            forecasts.append(forecast_df)
            
        # Run garbage collection to free memory
        gc.collect()
    
    return pd.concat(forecasts)

# Run the forecasting function
results_df = fit_forecast_sarima(df)

# Calculate overall error metrics
mse = mean_squared_error(results_df['Actual_Demand'], results_df['Predicted_Demand'])
mae = mean_absolute_error(results_df['Actual_Demand'], results_df['Predicted_Demand'])
rmse = np.sqrt(mse)

print(f"Overall MSE: {mse}")
print(f"Overall MAE: {mae}")
print(f"Overall RMSE: {rmse}")

# Write the results to a file
results_df['Date'] = results_df['Datetime'].dt.date
results_df['Hour'] = results_df['Datetime'].dt.hour
results_df = results_df[['ID', 'Date', 'Hour', 'Actual_Demand', 'Predicted_Demand']]
results_df.to_csv('sarima_results_1_5050.csv', index=False)

print("Forecast results have been written to forecast_results_2_8020.csv")


1 Processing ID: Exp_553
2 Processing ID: Exp_401
3 Processing ID: Exp_266
4 Processing ID: Exp_62
5 Processing ID: Exp_111
6 Processing ID: Exp_567
7 Processing ID: Exp_570
8 Processing ID: Exp_500
9 Processing ID: Exp_52
10 Processing ID: Exp_222
11 Processing ID: Exp_587
12 Processing ID: Exp_17
13 Processing ID: Exp_462
14 Processing ID: Exp_271
15 Processing ID: Exp_49
16 Processing ID: Exp_451
17 Processing ID: Exp_502
18 Processing ID: Exp_86
19 Processing ID: Exp_612
20 Processing ID: Exp_509
21 Processing ID: Exp_12
22 Processing ID: Exp_208
23 Processing ID: Exp_529
24 Processing ID: Exp_128
25 Processing ID: Exp_56
26 Processing ID: Exp_635
27 Processing ID: Exp_93
28 Processing ID: Exp_313
29 Processing ID: Exp_446
30 Processing ID: Exp_643
31 Processing ID: Exp_223
32 Processing ID: Exp_315
33 Processing ID: Exp_649
34 Processing ID: Exp_191
35 Processing ID: Exp_445
36 Processing ID: Exp_518
37 Processing ID: Exp_365
38 Processing ID: Exp_356
39 Processing ID: Exp_420
40 

In [2]:
import pandas as pd
import pmdarima as pm
from pmdarima import auto_arima
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np
import warnings
warnings.filterwarnings("ignore")

# Load your dataset (assuming it is a CSV file)
df = pd.read_csv('phase2_only_records.csv')

# Convert 'Date' and 'Hour' columns to a datetime object
df['Datetime'] = pd.to_datetime(df['Date']) + pd.to_timedelta(df['Hour'], unit='h')

# Set 'Datetime' as index
df.set_index('Datetime', inplace=True)
exog_vars = ['Temperature']

# Prepare a function to fit and forecast using auto_arima for each ID
def fit_forecast_arima(df, split_ratio=0.8):
    ids = df['ID'].unique()
    forecasts = []

    for i, id_ in enumerate(ids[:500], start=1):
        print(f"{i} Processing ID: {id_}")

        df_id = df[df['ID'] == id_].sort_index()

        # Split the data into train and test
        split_index = int(len(df_id) * split_ratio)
        train = df_id.iloc[:split_index]
        test = df_id.iloc[split_index:]

        # Fit the ARIMA model
        model = auto_arima(train['Demand_kWh'],
                           exogenous=train[exog_vars],
                           seasonal=True,
                           m=24,  # Assuming daily seasonality with hourly data
                           stepwise=True,
                           suppress_warnings=True,
                           trace=False,
                           max_p=2, max_q=2, max_P=1, max_Q=1, max_order=5)

        # Forecast
        n_periods = len(test)
        forecast = model.predict(n_periods=n_periods, exogenous=test[exog_vars])

        # Store the forecast and actual values
        forecast_df = pd.DataFrame({
            'ID': id_,
            'Datetime': test.index,
            'Actual_Demand': test['Demand_kWh'],
            'Predicted_Demand': forecast
        })

        forecasts.append(forecast_df)

    return pd.concat(forecasts)

# Run the forecasting function
results_df = fit_forecast_arima(df)

# Calculate overall error metrics
mse = mean_squared_error(results_df['Actual_Demand'], results_df['Predicted_Demand'])
mae = mean_absolute_error(results_df['Actual_Demand'], results_df['Predicted_Demand'])
rmse = np.sqrt(mse)

print(f"Overall MSE: {mse}")
print(f"Overall MAE: {mae}")
print(f"Overall RMSE: {rmse}")

# Write the results to a file
results_df['Date'] = results_df['Datetime'].dt.date
results_df['Hour'] = results_df['Datetime'].dt.hour
results_df = results_df[['ID', 'Date', 'Hour', 'Actual_Demand', 'Predicted_Demand']]
results_df.to_csv('sarima_results_2_8020.csv', index=False)

print("Forecast results have been written to forecast_results_2_8020.csv")

1 Processing ID: Exp_3928
2 Processing ID: Exp_3903
3 Processing ID: Exp_3921
4 Processing ID: Exp_3844
5 Processing ID: Exp_3911
6 Processing ID: Exp_3913
7 Processing ID: Exp_3896
8 Processing ID: Exp_3922
9 Processing ID: Exp_3919
10 Processing ID: Exp_3891
11 Processing ID: Exp_3917
12 Processing ID: Exp_3858
13 Processing ID: Exp_3884
14 Processing ID: Exp_3900
15 Processing ID: Exp_3934
16 Processing ID: Exp_3904
17 Processing ID: Exp_3901
18 Processing ID: Exp_3868
19 Processing ID: Exp_3889
20 Processing ID: Exp_3920
21 Processing ID: Exp_3867
22 Processing ID: Exp_3931
23 Processing ID: Exp_3880
24 Processing ID: Exp_3895
25 Processing ID: Exp_3860
26 Processing ID: Exp_3843
27 Processing ID: Exp_3873
28 Processing ID: Exp_3914
29 Processing ID: Exp_3883
30 Processing ID: Exp_3871
31 Processing ID: Exp_3924
32 Processing ID: Exp_3929
33 Processing ID: Exp_3837
34 Processing ID: Exp_3912
35 Processing ID: Exp_3935
36 Processing ID: Exp_3874
37 Processing ID: Exp_3877
38 Process